In [2]:
# import requirements
import numpy as np
import random
import os
import shutil
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# tensorflow imports 
from tensorflow.keras import models
from tensorflow.keras import Sequential, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Train_test_split

In [5]:
# # Creating Train / Val / Test folders

root_dir = '../raw_data/frames' # data root path
classes_dir = ['violence', 'non_violence'] #total labels

val_ratio = 0.15
test_ratio = 0.05

for cls in classes_dir:
    os.makedirs(root_dir +'train/' + cls)
    os.makedirs(root_dir +'val/' + cls)
    os.makedirs(root_dir +'test/' + cls)


# Creating partitions of the data after shuffling


    src = root_dir + "/" + cls # Folder to copy images from

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                          [int(len(allFileNames)* (1 - (val_ratio + test_ratio))), 
                                                           int(len(allFileNames)* (1 - test_ratio))])


    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))
    print('Testing: ', len(test_FileNames))

    # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, root_dir +'train/' + cls)

    for name in val_FileNames:
        shutil.copy(name, root_dir +'val/' + cls)

    for name in test_FileNames:
        shutil.copy(name, root_dir +'test/' + cls)

Total images:  5832
Training:  4665
Validation:  875
Testing:  292
Total images:  5231
Training:  4184
Validation:  785
Testing:  262


# Load Data with Augmentation

In [6]:
# import data
root = r'../raw_data/'

train_dir = root + "framestrain/"

train_datagen = ImageDataGenerator(rescale=1.0/255., 
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest'
                                    )

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=16,
                                                    classes=['violence', 
                                                             'non_violence' 
                                                             ],
                                                    class_mode="categorical",
                                                    color_mode="rgb",
                                                    target_size=(224, 224))

val_dir = root + "framesval/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)

validation_generator = validation_datagen.flow_from_directory(val_dir,
                                                              batch_size=16,
                                                                classes=['violence', 
                                                                         'non_violence'
                                                                        ],
                                                              class_mode="categorical",
                                                              color_mode="rgb",
                                                              target_size=(224, 224)
                                                             
                                                             )


Found 8849 images belonging to 2 classes.
Found 1660 images belonging to 2 classes.


# Transfer-learning

In [7]:
import tensorflow
from tensorflow.keras.applications.vgg19 import VGG19

def load_vgg19(dense_n=512, lr= 0.000134,):
    model = tensorflow.keras.applications.VGG19(
                                            include_top=False, weights="imagenet",
                                            input_shape=(224, 224, 3), pooling="max", classes=2,
    )
    model.trainable = False 
    model = tensorflow.keras.models.Sequential([ 
                            model,
                            tensorflow.keras.layers.Flatten(),
                            tensorflow.keras.layers.Dense(dense_n, activation='relu'),
                            tensorflow.keras.layers.Dense(1, activation='sigmoid')
                                ])
    
    opt = optimizers.Adam(learning_rate=lr)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [8]:
model = load_vgg19()

2021-08-21 15:21:25.943305: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 512)               20024384  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 20,287,553
Trainable params: 263,169
Non-trainable params: 20,024,384
_________________________________________________________________


In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=20, verbose=1, restore_best_weights=True)

model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs= 5,
                    verbose=1,
                    callbacks = es)